# Modelado aditivo por etapas hacia adelante

**Forward stagewise additive modeling (FSAM)** optimiza la pérdida empı́rica con pérdida general y $f$ modelo aditivo. En la iteración $m$ calcula:
$$(\beta_m,\boldsymbol{\theta}_m)%
=\operatorname*{argmin}_{\beta,\boldsymbol{\theta}}\;L_m(\beta,\boldsymbol{\theta})%
\qquad\text{con}\qquad%
L_m(\beta,\boldsymbol{\theta})%
=\sum_{i=1}^N\ell(y_i, f_{m-1}(\boldsymbol{x}_i)+\beta F(\boldsymbol{x}_i;\boldsymbol{\theta}))$$
y reajusta el modelo,
$$f_m(\boldsymbol{x})=f_{m-1}(\boldsymbol{x})+\beta_m F_m(\boldsymbol{x})%
\qquad\text{con}\qquad%
F_m(\boldsymbol{x})=F(\boldsymbol{x};\boldsymbol{\theta}_m)$$

## Pérdida cuadrática y boosting mínimos cuadrados

Si $\ell(y,\hat{y})=(y-\hat{y})^2$, el objetivo en la iteración $m$ es:
$$L_m(\beta,\boldsymbol{\theta})%
=\sum_{i=1}^N (r_{im}-\beta F(\boldsymbol{x}_i;\boldsymbol{\theta}))^2
\qquad\text{con}\qquad r_{im}=y_i-f_{m-1}(\boldsymbol{x}_i)$$
Boosting mínimos cuadrados minimiza el objetivo fijando $\beta=1$ y ajustando $F$ a los residuos.

## Pérdida exponencial y AdaBoost

Si $\ell(\tilde{y},\hat{y})=\exp(-\tilde{y}\hat{y})$, $\tilde{y}\in\{-1,+1\}$, el objetivo en la iteración $m$ es:
$$\begin{align*}
L_m(\beta,\boldsymbol{\theta})%
&=\sum_{i=1}^N \exp(-\tilde{y}_i(f_{m-1}(\boldsymbol{x}_i)+\beta F(\boldsymbol{x}_i;\boldsymbol{\theta})))\\%
&=\sum_{i=1}^N w_{im}\exp(-\beta\tilde{y}_i F(\boldsymbol{x}_i;\boldsymbol{\theta}))%
\qquad\text{con}\qquad w_{im}=\exp(-\tilde{y}_i f_{m-1}(\boldsymbol{x}_i))\\%
&=e^{-\beta}\sum_{\tilde{y}_i=F(\boldsymbol{x}_i;\boldsymbol{\theta})} w_{im}%
+e^{\beta}\sum_{\tilde{y}_i\neq F(\boldsymbol{x}_i;\boldsymbol{\theta})} w_{im}\\%
&=(e^{\beta}-e^{-\beta})\sum_{i=1}^N w_{im}\mathbb{I}(\tilde{y}_i\neq F(\boldsymbol{x}_i;\boldsymbol{\theta}))%
+e^{-\beta}\sum_{i=1}^N w_{im}
\end{align*}$$